<a href="https://colab.research.google.com/github/KhomZ/tensorflow-with-khom/blob/main/save_and_load_models_in_tf_khom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TensorFlow Core**


```
# TensorFlow with Khom
```

*Save and load models*

Model progress can be saved during and after training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:

* code to create the model, and
* the trained weights, or parameters, for the model

Sharing this data helps others understand how the model works and try it themselves with new data.

Caution: TensorFlow models are code and it is important to be careful with untrusted code. See Using TensorFlow Securely for details.

Options
There are different ways to save TensorFlow models depending on the API you're using. This guide uses tf.keras—a high-level API to build and train models in TensorFlow. For other approaches, refer to the Using the SavedModel format guide and the Save and load Keras models guide.

In [2]:
# Setup
# install and import TensorFlow and dependencies:
!pip install pyyaml h5py  # Required to save models in HDF5 format


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.8.2


In [4]:
# TensorFlow with khom
# save and load model

# Get an example dataset
# To demonstrate how to save and load weights, you'll use the MNIST dataset. 
# To speed up these runs, use the first 1000 examples:

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [5]:
# Define a model
# Start by building a simple sequential model:

def create_model():
  model = tf.keras.Sequential([
                               keras.layers.Dense(512, activation='relu', input_shape=(784,)),
                               keras.layers.Dropout(0.2),
                               keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])
  
  return model


In [7]:
# Create a basic model instance
model = create_model()
  

In [8]:
# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Save checkpoints during training
You can use a trained model without having to retrain it, or pick-up training where you left off in case the training process was interrupted. The tf.keras.callbacks.ModelCheckpoint callback allows you to continually save the model both during and at the end of training.

Checkpoint callback usage
Create a tf.keras.callbacks.ModelCheckpoint callback that saves weights only during training:

In [9]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [11]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [12]:
# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
27/32 [========================>.....] - ETA: 0s - loss: 1.2174 - sparse_categorical_accuracy: 0.6644
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 17ms/step - loss: 1.1550 - sparse_categorical_accuracy: 0.6800 - val_loss: 0.7724 - val_sparse_categorical_accuracy: 0.7650
Epoch 2/10
29/32 [==========================>...] - ETA: 0s - loss: 0.4405 - sparse_categorical_accuracy: 0.8707
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.4324 - sparse_categorical_accuracy: 0.8750 - val_loss: 0.5527 - val_sparse_categorical_accuracy: 0.8150
Epoch 3/10
28/32 [=========================>....] - ETA: 0s - loss: 0.3106 - sparse_categorical_accuracy: 0.9185
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.3082 - sparse_categorical_accuracy: 0.9150 - val_loss: 0.4825 - val_sparse_categorical_accuracy: 0.8390
Epoch 4/10
32/32 [=========

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [13]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'checkpoint', 'cp.ckpt.index']



```
['cp.ckpt.index', 'cp.ckpt.data-00000-of-00001', 'checkpoint']
```



As long as two models share the same architecture you can share weights between them. So, when restoring a model from weights-only, create a model with the same architecture as the original model and then set its weights.

Now rebuild a fresh, untrained model and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [14]:
# Create a basic model instance
model = create_model()

In [15]:
# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3453 - sparse_categorical_accuracy: 0.0760 - 155ms/epoch - 5ms/step
Untrained model, accuracy:  7.60%


Then load the weights from the checkpoint and re-evaluate:

In [16]:
# Loads the weights
model.load_weights(checkpoint_path)

In [17]:
# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4025 - sparse_categorical_accuracy: 0.8760 - 69ms/epoch - 2ms/step
Restored model, accuracy: 87.60%


Checkpoint callback options


The callback provides several options to provide unique names for checkpoints and adjust the checkpointing frequency.

Train a new model, and save uniquely named checkpoints once every five epochs:

In [18]:
# Include the epoch in the file name (uses 'str.format')
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

In [19]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size
)

In [20]:
# Create a new model instance
model = create_model()

In [21]:
# Save the weights using the 'checkpoint_path' format
model.save_weights(checkpoint_path.format(epoch=0))

In [22]:
# Train the model with the new callback
model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


Now, review the resulting checkpoints and choose the latest one:

In [23]:
os.listdir(checkpoint_dir)

['cp-0025.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0000.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'checkpoint',
 'cp-0045.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001']

In [24]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'



```
Note: The default TensorFlow format only saves the 5 most recent checkpoints.
```



To test, reset the model, and load the latest checkpoint:

In [25]:
# Create a new model instance
model = create_model()

In [26]:
# Load the previously saved weights
model.load_weights(latest)

In [27]:
# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4831 - sparse_categorical_accuracy: 0.8860 - 164ms/epoch - 5ms/step
Restored model, accuracy: 88.60%


What are these files?

The above code stores the weights to a collection of checkpoint-formatted files that contain only the trained weights in a binary format. Checkpoints contain:

One or more shards that contain your model's weights.
An index file that indicates which weights are stored in which shard.
If you are training a model on a single machine, you'll have one shard with the suffix: `.data-00000-of-00001`

**Manually save weights**

To save weights manually, use tf.keras.Model.save_weights. By default, tf.keras—and the Model.save_weights method in particular—uses the TensorFlow Checkpoint format with a .ckpt extension. To save in the HDF5 format with a .h5 extension, refer to the Save and load models guide.

In [28]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

In [29]:
# Create a new model instance
model = create_model()

In [30]:
# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

In [31]:
# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4831 - sparse_categorical_accuracy: 0.8860 - 164ms/epoch - 5ms/step
Restored model, accuracy: 88.60%


**Save the entire model**

Call tf.keras.Model.save to save a model's architecture, weights, and training configuration in a single file/folder. This allows you to export a model so it can be used without access to the original Python code*. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

An entire model can be saved in two different file formats (SavedModel and HDF5). The TensorFlow SavedModel format is the default file format in TF2.x. However, models can be saved in HDF5 format. More details on saving entire models in the two file formats is described below.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js (Saved Model, HDF5) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite (Saved Model, HDF5)

*Custom objects (for example, subclassed models or layers) require special attention when saving and loading. Refer to the Saving custom objects section below.

**SavedModel format**

The SavedModel format is another way to serialize models. Models saved in this format can be restored using tf.keras.models.load_model and are compatible with TensorFlow Serving. The SavedModel guide goes into detail about how to serve/inspect the SavedModel. The section below illustrates the steps to save and restore the model.

In [32]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1765 - sparse_categorical_accuracy: 0.6810
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4246 - sparse_categorical_accuracy: 0.8790
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.3009 - sparse_categorical_accuracy: 0.9170
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2105 - sparse_categorical_accuracy: 0.9530
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1538 - sparse_categorical_accuracy: 0.9650


In [33]:
# Save the entire model as a SaveModel.
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


The SavedModel format is a directory containing a protobuf binary and a TensorFlow checkpoint. Inspect the saved model directory:

In [38]:
ls

checkpoints/  sample_data/  saved_model/  training_1/  training_2/


In [39]:
# my_model directory
ls saved_model

my_model/


In [40]:
# Contains an assets folder, saved_model.pb, and variables folder.
ls saved_model/my_model

assets/  keras_metadata.pb  saved_model.pb  variables/


Reload a fresh Keras model from the saved model:

In [42]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


The restored model is compiled with the same arguments as the original model. Try running evaluate and predict with the loaded model:

In [43]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4190 - sparse_categorical_accuracy: 0.8640 - 171ms/epoch - 5ms/step
Restored model, accuracy: 86.40%
(1000, 10)


**HDF5 format**

Keras provides a basic save format using the HDF5 standard.

In [44]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.1448 - sparse_categorical_accuracy: 0.6800
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4380 - sparse_categorical_accuracy: 0.8780
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2914 - sparse_categorical_accuracy: 0.9220
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2146 - sparse_categorical_accuracy: 0.9470
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1609 - sparse_categorical_accuracy: 0.9670


In [45]:
# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5')

Now, recreate the model from that file:

In [46]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Check its accuracy:

In [47]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4219 - sparse_categorical_accuracy: 0.8640 - 202ms/epoch - 6ms/step
Restored model, accuracy: 86.40%


Keras saves models by inspecting their architectures. This technique saves everything:

* The weight values
* The model's architecture
* The model's training configuration (what you pass to the .compile() method)
* The optimizer and its state, if any (this enables you to restart training where you left off)

Keras is not able to save the v1.x optimizers (from tf.compat.v1.train) since they aren't compatible with checkpoints. For v1.x optimizers, you need to re-compile the model after loading—losing the state of the optimizer.

**Saving custom objects**

If you are using the SavedModel format, you can skip this section. The key difference between HDF5 and SavedModel is that HDF5 uses object configs to save the model architecture, while SavedModel saves the execution graph. Thus, SavedModels are able to save custom objects like subclassed models and custom layers without requiring the original code.

To save custom objects to HDF5, you must do the following:

1.   Define a `get_config` method in your object, and optionally a `from_config` classmethod.

*   `get_config(self)` returns a JSON-serializable dictionary of parameters needed to recreate the object.
*   `from_config(cls, config)` uses the returned config from `get_config` to create a new object. By default, this function will use the config as initialization kwargs `(return cls(**config))`.


2.   Pass the object to the `custom_objects` argument when loading the model. The argument must be a dictionary mapping the string class name to the Python class. E.g. `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`

Refer to the Writing layers and models from scratch tutorial for examples of custom objects and `get_config.`

In [ ]:
# Khom 
# @author: Khom